In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import os
import numpy
from PIL import Image
import torch
import numpy as np
from torch import nn
from tqdm.auto import tqdm
from torchvision import transforms
from torchvision.datasets import ImageFolder
from torchvision.utils import make_grid
from torch.utils.data import DataLoader
import torch.nn.functional as F
import matplotlib.pyplot as plt
import matplotlib.image as implt
import pandas as pd
import  glob
import random

In [ ]:
os.rmdir('.virtual_documents') 

In [ ]:
class Generator(nn.Module):
    '''
    Generator Class
    Values:
        z_dim: the dimension of the noise vector, a scalar
        im_chan: the number of channels of the output image, a scalar
              (MNIST is black-and-white, so 1 channel is your default)
        hidden_dim: the inner dimension, a scalar
    '''
    def __init__(self, input_dim=171, im_chan=3, hidden_dim=64):
        super(Generator, self).__init__()
        self.input_dim = input_dim
        # Build the neural network
        self.gen = nn.Sequential(
            self.make_gen_block(input_dim, hidden_dim * 4),
            self.make_gen_block(hidden_dim * 4, hidden_dim * 2, kernel_size=4, stride=1),
            self.make_gen_block(hidden_dim * 2, hidden_dim),
            self.make_gen_block(hidden_dim, im_chan, kernel_size=4, final_layer=True),
        )

    def make_gen_block(self, input_channels, output_channels, kernel_size=3, stride=2, final_layer=False):
        '''
        Function to return a sequence of operations corresponding to a generator block of DCGAN;
        a transposed convolution, a batchnorm (except in the final layer), and an activation.
        Parameters:
            input_channels: how many channels the input feature representation has
            output_channels: how many channels the output feature representation should have
            kernel_size: the size of each convolutional filter, equivalent to (kernel_size, kernel_size)
            stride: the stride of the convolution
            final_layer: a boolean, true if it is the final layer and false otherwise 
                      (affects activation and batchnorm)
        '''
        if not final_layer:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.BatchNorm2d(output_channels),
                nn.ReLU(inplace=True),
            )
        else:
            return nn.Sequential(
                nn.ConvTranspose2d(input_channels, output_channels, kernel_size, stride),
                nn.Tanh(),
            )

    def forward(self, noise):
        '''
        Function for completing a forward pass of the generator: Given a noise tensor,
        returns generated images.
        Parameters:
            noise: a noise tensor with dimensions (n_samples, z_dim)
        '''
        x = noise.view(len(noise), self.input_dim, 1, 1)
        return self.gen(x)

In [ ]:
def get_one_hot_labels(labels, n_classes=43):
    return F.one_hot(labels, n_classes)

def combine_vectors(x, y):
    return torch.cat((x.float(), y.float()), 1)

def get_input_dimensions(z_dim, img_shape=(3, 64, 64), n_classes=43):
    gen_input_dim = z_dim + n_classes
    disc_input_dim = img_shape[0] + n_classes
    
    return gen_input_dim, disc_input_dim

def get_noise(n_samples, z_dim, device='cpu'):
    '''
    Function for creating noise vectors: Given the dimensions (n_samples, z_dim)
    creates a tensor of that shape filled with random numbers from the normal distribution.
    Parameters:
      n_samples: the number of samples to generate, a scalar
      z_dim: the dimension of the noise vector, a scalar
      device: the device type
    '''
    return torch.randn(n_samples, z_dim, device=device)

In [ ]:
z_dim = 128
gen_in_dim, _ = get_input_dimensions(z_dim)

In [ ]:
device='cuda'
gen = Generator(input_dim=gen_in_dim).to(device)

In [ ]:
def show_tensor_images(image_tensor, num_images=25, size=(1, 64, 64)):
    '''
    Function for visualizing images: Given a tensor of images, number of images, and
    size per image, plots and prints the images in an uniform grid.
    '''
    image_tensor = (image_tensor + 1) / 2
    image_unflat = image_tensor.detach().cpu()
    image_grid = make_grid(image_unflat[:num_images], nrow=5)
    plt.imshow(image_grid.permute(1, 2, 0).squeeze())
    plt.show()
    
def make_grad_hook():
    '''
    Function to keep track of gradients for visualization purposes, 
    which fills the grads list when using model.apply(grad_hook).
    '''
    grads = []
    def grad_hook(m):
        if isinstance(m, nn.Conv2d) or isinstance(m, nn.ConvTranspose2d):
            grads.append(m.weight.grad)
    return grads, grad_hook


In [ ]:
classes = { 0:'Speed limit (20km/h)', 
            1:'Speed limit (30km/h)', 
            2:'Speed limit (50km/h)', 
            3:'Speed limit (60km/h)', 
            4:'Speed limit (70km/h)', 
            5:'Speed limit (80km/h)', 
            6:'End of speed limit (80km/h)', 
            7:'Speed limit (100km/h)', 
            8:'Speed limit (120km/h)', 
            9:'No passing', 
            10:'No passing veh over 3.5 tons', 
            11:'Right-of-way at intersection', 
            12:'Priority road', 
            13:'Yield', 
            14:'Stop', 
            15:'No vehicles', 
            16:'Veh > 3.5 tons prohibited', 
            17:'No entry', 
            18:'General caution', 
            19:'Dangerous curve left', 
            20:'Dangerous curve right', 
            21:'Double curve', 
            22:'Bumpy road', 
            23:'Slippery road', 
            24:'Road narrows on the right', 
            25:'Road work', 
            26:'Traffic signals', 
            27:'Pedestrians', 
            28:'Children crossing', 
            29:'Bicycles crossing', 
            30:'Beware of ice/snow',
            31:'Wild animals crossing', 
            32:'End speed + passing limits', 
            33:'Turn right ahead', 
            34:'Turn left ahead', 
            35:'Ahead only', 
            36:'Go straight or right', 
            37:'Go straight or left', 
            38:'Keep right', 
            39:'Keep left', 
            40:'Roundabout mandatory', 
            41:'End of no passing', 
            42:'End no passing veh > 3.5 tons' }

In [ ]:
gen_weights = torch.load('/kaggle/input/gan-model/generator.pth')
gen.load_state_dict(gen_weights)
gen.eval()
count = 1500*43

labels=[]
for i in range(43):
    labels += [i]*1500


test_noise = get_noise(count, z_dim, device)
test_label = torch.tensor(labels)
test_OH_label = get_one_hot_labels(test_label.to(device))
test_noise_and_label = combine_vectors(test_noise, test_OH_label)
test_image_fake = gen(test_noise_and_label)

In [ ]:
for i in classes.values():
    os.mkdir(i.replace('/',''))

In [ ]:
test_df = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')

In [ ]:
for index,i in enumerate(test_image_fake):
    numpy_image = i.to('cpu').detach().numpy()
    numpy_image = (np.moveaxis(numpy_image, 0,-1)*128 + 128).astype(np.uint8)
    plt.imsave(os.path.join('/kaggle/working/',classes[index//1500].replace('/',''),str(index%1500)) + '.jpg', numpy_image)

In [ ]:
for className in os.listdir('/kaggle/input/gtsrb-german-traffic-sign/Train'):
    classPath = os.path.join('/kaggle/input/gtsrb-german-traffic-sign/Train',className)
    for index, imageName in enumerate(os.listdir(classPath)):
        img = implt.imread(os.path.join(classPath, imageName))
        outputPath = os.path.join('/kaggle/working/', classes[int(className)].replace('/',''), str(1500+index)) + '.jpg'
        plt.imsave(outputPath, img)

In [ ]:
for index, imageName in enumerate(os.listdir('/kaggle/input/gtsrb-german-traffic-sign/Test')):
    if imageName == 'GT-final_test.csv':
        continue
    try:
        imagePath = os.path.join('/kaggle/input/gtsrb-german-traffic-sign/Test', imageName)
        img = implt.imread(imagePath)
        classId = (test_df[test_df["Path"] == 'Test/' + imageName]['ClassId']).iloc[0]
        outputPath = os.path.join('/kaggle/working/', classes[classId].replace('/',''), str(1860+index)) + '.jpg'
        plt.imsave(outputPath, img)
    except:
        print('Error:')
        print(imageName)
        print(imagePath)

In [ ]:
import shutil
shutil.make_archive('/kaggle/working/generated_images', 'zip', '/kaggle/working/')